In [1]:
from ortools.sat.python import cp_model
import pandas as pd

In [2]:
# Read CSV
class_list = pd.read_csv("class_list.csv")
timeslots_days = pd.read_csv("timeslots.csv")

lessons_required = pd.read_csv("lessons_required.csv")
weekday = pd.read_csv("weekdays.csv")
teacher_classes = pd.read_csv("teacher_classes.csv")
student_classes = pd.read_csv("student_classes.csv")

teacher_list = pd.read_csv("teacher_list.csv")

In [3]:
student_classes = student_classes[student_classes['class']!='isizulu-12']
teacher_classes = teacher_classes[teacher_classes['class']!='isizulu-12']

In [4]:
classes = list(class_list['class'])
timeslots = list(timeslots_days['timeslot'])
days = list(weekday['weekday'])
periods = range(9)
teacher = list(teacher_list['teacher'])

num_lessons = dict(zip(lessons_required["class"], lessons_required["num_lessons"]))
class_teacher = dict(zip(teacher_classes["class"], teacher_classes["teacher"]))
class_students = dict(zip(student_classes["class"], student_classes["student"]))

In [5]:
# # Build student → classes mapping
# student_classes_map = {}
# for _, row in student_classes.iterrows():
#     student = row['student']
#     cls = row['class']
#     if student not in student_classes_map:
#         student_classes_map[student] = []
#     student_classes_map[student].append(cls)

In [6]:
# # Build student → classes mapping
# teacher_classes_map = {}
# for _, row in teacher_classes.iterrows():
#     teacher = row['teacher']
#     cls = row['class']
#     if teacher not in teacher_classes_map:
#         teacher_classes_map[teacher] = []
#     teacher_classes_map[teacher].append(cls)

In [7]:
model = cp_model.CpModel()

## Variables

We’ll create one **binary decision variable** per `(class, day, period)`

In [8]:
# Variables
x = {}
for c in classes:
    for d in days:
        for p in periods:
            x[(c,d,p)] = model.NewBoolVar(f'x_{c}_{d}_{p}')

## Constraints

### Required lessons per class

Each class must be scheduled the exact number of times specified in `num_lessons`.

In [9]:
# Constraint: each class gets required lessons
for c in classes:
    model.Add(sum(x[(c,d,p)] for d in days for p in periods) == num_lessons[c])

### Teachers cannot be double-booked

For each `(day, period)`, a teacher can only teach one of their classes.

In [10]:
for d in days:
    for p in periods:
        for t in set(class_teacher.values()):  # each teacher
            teaching_classes = [cls for cls, teacher in class_teacher.items() if teacher == t]
            if teaching_classes:
                model.Add(sum(x[(c,d,p)] for c in teaching_classes if c in classes) <= 1)

In [11]:
# for d in days:
#     for p in periods:
#         for s, enrolled_classes in teacher_classes_map.items():
#             model.Add(sum(x[(c,d,p)] for c in enrolled_classes) <= 1)

### Students cannot be double-booked

Same idea: For each `(day, period)`, a student can only attend one class.

In [12]:
for d in days:
    for p in periods:
        for s in set(class_students.values()):
            enrolled_classes = [c for c, students in class_students.items() if s in students]
            if enrolled_classes:
                model.Add(sum(x[(c,d,p)] for c in enrolled_classes if c in classes) <= 1)

In [13]:
# for d in days:
#     for p in periods:
#         for s, enrolled_classes in student_classes_map.items():
#             model.Add(sum(x[(c,d,p)] for c in enrolled_classes) <= 1)

### Spread lessons across days (soft preference)

For now, we’ll keep it simple, but later we can add an **objective** to penalize imbalance.

In [14]:
for c in classes:
    lessons = num_lessons[c]
    for d in days:
        if lessons <= 5:
            # Spread: at most 1 per day
            model.Add(sum(x[(c, d, p)] for p in periods) <= 1)
        else:
            # Allow 2 per day if more than 5 lessons in total
            model.Add(sum(x[(c, d, p)] for p in periods) <= 2)

In [15]:
for c in classes:
    for d in days:
        # List of variables for this class on this day
        vars_day = [x[(c, d, p)] for p in periods]

        # If 2 lessons in this day, they must be consecutive
        # Create pairwise "consecutive slot" indicators
        consecutive_pairs = []
        for p in range(len(periods) - 1):
            pair = model.NewBoolVar(f"{c}_{d}_consec_{p}")
            model.Add(vars_day[p] + vars_day[p+1] == 2).OnlyEnforceIf(pair)
            model.Add(vars_day[p] + vars_day[p+1] != 2).OnlyEnforceIf(pair.Not())
            consecutive_pairs.append(pair)

        # Now enforce: if there are 2 lessons, at least one consecutive pair is active
        is_two = model.NewBoolVar(f"{c}_{d}_is_two")
        model.Add(sum(vars_day) == 2).OnlyEnforceIf(is_two)
        model.Add(sum(vars_day) != 2).OnlyEnforceIf(is_two.Not())

        if consecutive_pairs:
            model.Add(sum(consecutive_pairs) >= 1).OnlyEnforceIf(is_two)

## Step 3. Solver & Output

In [16]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 20
status = solver.Solve(model)

In [17]:
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    rows = []
    for (c, d, p), var in x.items():
        if solver.Value(var) == 1:
            rows.append([d, p, c])
    output = pd.DataFrame(rows, columns=['weekday', 'period', 'class'])
    print("Solution written to timetable_solution.csv")
else:
    print("No feasible solution found")

Solution written to timetable_solution.csv


In [18]:
period_interval = timeslots_days.copy()

# Split on ' - '
split_cols = period_interval['timeslot'].str.rsplit(' - ', n=1, expand=True)

# Clean start_time (remove weekday, replace '_' with ':')
period_interval['start_time'] = (
    split_cols[0]
    .str.replace(r'^[A-Z]+\s*', '', regex=True)  # drop weekday like 'FRI'
    .str.replace('_', ':', regex=False)          # turn 07_45 -> 07:45
    .str.strip()
)

# Get unique start times in order of appearance
unique_times = period_interval['start_time'].drop_duplicates().tolist()

period_start_times = pd.DataFrame({
    "period": range(len(unique_times)),
    "start_time": unique_times
})

period_interval = period_interval.merge(period_start_times, on="start_time", how="left")

In [19]:
final = output.merge(period_interval, on = ['period', 'weekday'], how  ='left')[['timeslot', 'class']]

In [20]:
final.to_csv("timetable4.csv", index=False)

In [21]:
# python opt_test.py timetable3.csv

In [22]:
# Check if there is any missing catogorise, consistency of categories in every dataset (techers, students, subjects/class, periods, weekdays)

# check for data quality (check duplicates, nulls, etc)
# check for inconsistencies


# # Read CSV
# class_list = pd.read_csv("class_list.csv")
# lessons_required = pd.read_csv("lessons_required.csv")
# student_classes = pd.read_csv("student_classes.csv")
# student_list = pd.read_csv("student_list.csv")
# teacher_classes = pd.read_csv("teacher_classes.csv")
# teacher_list = pd.read_csv("teacher_list.csv")
# timeslots_days = pd.read_csv("timeslots.csv")
# weekday = pd.read_csv("weekdays.csv")




# class_list.head()

# class
# 0	accn-10
# 1	accn-11
# 2	accn-12
# 3	bstd-10
# 4	bstd-11


# lessons_required.head()

# 	class	num_lessons
# 0	english-10	7
# 1	afrikaans-10	5
# 2	isizulu-10	7
# 3	lo-10	2
# 4	math-10	6


# student_classes.head()

# student	class
# 0	Emma Ortega	accn-10
# 1	Alex Lopez	accn-10
# 2	Mawhiba el-Moghaddam	accn-10
# 3	Tia Shaye Taylor	accn-10
# 4	Brittany Zimmerman	accn-10

# student_list.head()


# student
# 0	Emma Ortega
# 1	Alex Lopez
# 2	Mawhiba el-Moghaddam
# 3	Tia Shaye Taylor
# 4	Brittany Zimmerman



# teacher_classes.head()

# teacher	class
# 0	Zuhra al-Bina	geog-12
# 1	Kevin Joh	grds-10
# 2	Aubry Rucker	lo-10
# 3	Ena Singleton	cstd-10
# 4	Padideh Park	inft-10



# teacher_list

# teacher
# 0	Zuhra al-Bina
# 1	Kevin Joh
# 2	Aubry Rucker
# 3	Ena Singleton
# 4	Padideh Park


# timeslots_days.head()

# timeslot	weekday
# 0	FRI 07_45 - 08_30	FRI
# 1	FRI 08_30 - 09_15	FRI
# 2	FRI 09_15 - 10_00	FRI
# 3	FRI 10_00 - 10_45	FRI
# 4	FRI 10_45 - 11_25	FRI



# weekday.head()

# 	weekday
# 0	MON
# 1	TUE
# 2	WED
# 3	THU
# 4	FRI
